In [44]:
import pandas as pd 
import scipy.stats as st
import numpy as np


# Mostrar todo el contenido de las celdas (sin recortes)
pd.set_option('display.max_colwidth', None)

# Mostrar todas las columnas 
pd.set_option('display.max_columns', None)

In [45]:
ruta = "/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/clasificacion_mia2025.csv"

dp = pd.read_csv(ruta)

In [46]:
dp = dp.drop(columns=["Unnamed: 0"])

dp["store_name"] = dp["store_name"].replace({"PRINCIPAL COTA":"MOSTRADOR COTA"})

In [47]:
stock_seguridad = {
    "SUCURSAL CALLE 6": 50.00,
    "SUCURSAL VALLADOLID": 50.00,
    "SUCURSAL NORTE": 50.00,
    "SUCURSAL MEDELLIN": 150.00,
    "MOSTRADOR COTA": 300.00,
    "SUCURSAL BARRANQUILLA": 150.00,
    "SUCURSAL BUCARAMANGA": 150.00,
    "SUCURSAL CALI": 150.00,
   # "ANTIOQUEÑA DE LUBRICANTES SGP SAS": 300.00
}

reorden_ideal = {
    "SUCURSAL CALLE 6": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL VALLADOLID": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL NORTE": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL MEDELLIN": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    #"ANTIOQUEÑA DE LUBRICANTES SGP SAS": {"Punto de reorden": 5.00, "Inventario ideal": 7.00},
    "SUCURSAL BARRANQUILLA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BUCARAMANGA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "MOSTRADOR COTA": {"Punto de reorden": 2.50, "Inventario ideal": 3.00},
    "SUCURSAL CALI": {"Punto de reorden": 7.00, "Inventario ideal": 9.00}, # Tiene para 7 semnaas de inventario , y cubre el inventario 9 semanas
}


In [48]:
dp.head(10)

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio
0,MOSTRADOR COTA,DCS10536137,2025.0,31,26.967491,51663.09,1393223.91,0.0080,0.0080,8.359343e+05,0.60,0.00,4.838710,0.60,1.0,0.22,AAA,0.999
1,MOSTRADOR COTA,BCE00606125,2025.0,32,25.719665,20564.31,528907.16,0.0030,0.0752,8.755933e+11,1655476.38,0.01,0.000000,1655476.98,0.0,0.03,AAA,0.999
2,MOSTRADOR COTA,DCS10536137,2025.0,32,24.270742,51663.09,1253901.53,0.0072,0.0152,5.391777e+05,0.43,0.01,3.214286,1655477.41,1.0,0.23,AAA,0.999
3,MOSTRADOR COTA,DCS10536137,2025.0,28,23.052800,51663.09,1190978.88,0.0069,0.0221,2.870259e+06,2.41,0.02,2.000000,1655479.82,0.8,0.20,AAA,0.999
4,MOSTRADOR COTA,DCS10536137,2025.0,33,21.843668,51663.09,1128511.39,0.0065,0.0286,5.529706e+05,0.49,0.02,3.750000,1655480.31,1.0,0.24,AAA,0.999
5,MOSTRADOR COTA,DCS10536137,2025.0,29,20.947520,51663.09,1082213.61,0.0062,0.0348,4.761740e+05,0.44,0.03,2.950820,1655480.75,0.8,0.20,AAA,0.999
6,MOSTRADOR COTA,BCE00606125,2025.0,33,20.575732,20564.31,423125.73,0.0024,0.0907,2.073316e+05,0.49,0.03,7.000000,1655481.24,1.0,0.26,AAA,0.999
7,MOSTRADOR COTA,DCS10536137,2025.0,30,18.852768,51663.09,973992.25,0.0056,0.0404,5.259558e+05,0.54,0.03,3.870968,1655481.78,1.0,0.25,AAA,0.999
8,MOSTRADOR COTA,DCS10536137,2025.0,34,17.474934,51663.09,902809.09,0.0052,0.0456,4.694607e+05,0.52,0.04,4.285714,1655482.30,1.0,0.26,AAA,0.999
9,MOSTRADOR COTA,BCE00606125,2025.0,34,16.460586,20564.31,338500.59,0.0020,0.1093,2.031004e+05,0.60,0.04,8.000000,1655482.90,1.0,0.27,AAA,0.999


## INVENTARIO DE SEGURIDAD

In [49]:
# Copia del df original
df_std = dp.copy()

# Ordenar datos por tienda, producto, año y semana
df_std = df_std.sort_values(by=["store_name", "product_ref", "Año", "semana"])

# Calcular la desviación móvil (rolling) de EMA con ventana de 12 semanas
df_std["Desviacion_EMA_calc"] = (
    df_std.groupby(["store_name", "product_ref"])["Ema"]
    .transform(lambda x: x.rolling(window=12, min_periods=1).std())
)

# Calcular también la media móvil del EMA (opcional, para validación)
df_std["Ema_media"] = (
    df_std.groupby(["store_name", "product_ref"])["Ema"]
    .transform(lambda x: x.rolling(window=12, min_periods=1).mean())
)

# === INVENTARIO IDEAL y PUNTO DE REORDEN ===
# Diccionario con los dos parámetros (en semanas de cobertura)
reorden_ideal = {
    "SUCURSAL CALLE 6": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL VALLADOLID": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL NORTE": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL MEDELLIN": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BARRANQUILLA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BUCARAMANGA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "MOSTRADOR COTA": {"Punto de reorden": 2.50, "Inventario ideal": 3.00},
    "SUCURSAL CALI": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
}

# Extraer columnas de reorden e ideal
df_std["lt_reorden"] = df_std["store_name"].map(lambda x: reorden_ideal.get(x, {}).get("Punto de reorden", np.nan))
df_std["lt_ideal"] = df_std["store_name"].map(lambda x: reorden_ideal.get(x, {}).get("Inventario ideal", np.nan))


# Calcular Z-score según nivel de servicio
df_std["Z"] = df_std["Nivel de servicio"].apply(lambda x: st.norm.ppf(x))

# === STOCK DE SEGURIDAD (rolling 12 semanas) ===
df_std["inventario_seguridad"] = (
    df_std["Z"] * df_std["Desviacion_EMA_calc"] * np.sqrt(df_std["lt_ideal"])
)

# Inventario ideal (nivel máximo recomendado)
df_std["inventario_ideal"] = (df_std["Ema"] * df_std["lt_ideal"]) + df_std["inventario_seguridad"]


# Punto de reorden (cuándo pedir)
df_std["punto_reorden"] = (df_std["Ema"] * df_std["lt_reorden"]) + df_std["inventario_seguridad"]



# === Redondear columnas numéricas ===
df_std["inventario_seguridad"] = df_std["inventario_seguridad"].round(2)
df_std["inventario_ideal"] = df_std["inventario_ideal"].round(2)
df_std["punto_reorden"] = df_std["punto_reorden"].round(2)




df_std = df_std.drop(columns=["Desviacion_EMA_calc","Ema_media","lt_reorden","lt_ideal","Z"])

# TARDA 2 MINUTOS 

In [50]:
df_std[(df_std["semana"]==40) & (df_std["store_name"]=="SUCURSAL BARRANQUILLA")].sort_values(by=["Ema"],ascending=False).head(10)

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio,inventario_seguridad,inventario_ideal,punto_reorden
54122,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,40,42.592951,13645.86,581217.45,0.0006,0.1633,1.104313e+05,0.19,0.02,4.079618,2.19,1.0,0.30,AAA,0.999,85.72,469.06,383.87
54152,SUCURSAL BARRANQUILLA,DAB14570025,2025.0,40,32.177602,9917.17,319110.75,0.0003,0.3160,1.244532e+05,0.39,0.06,4.935000,8.52,1.0,0.38,AAA,0.999,99.95,389.55,325.19
54154,SUCURSAL BARRANQUILLA,BLS00037125,2025.0,40,31.648724,32494.89,1028421.81,0.0011,0.0907,1.131264e+05,0.11,0.06,1.983806,8.75,0.8,0.25,AAA,0.999,27.76,312.60,249.30
54155,SUCURSAL BARRANQUILLA,BCS00035125,2025.0,40,30.682268,20581.34,631482.19,0.0007,0.1382,1.073520e+05,0.17,0.06,3.242105,8.92,1.0,0.31,AAA,0.999,39.71,315.85,254.49
54177,SUCURSAL BARRANQUILLA,BCS00025125,2025.0,40,26.604596,31057.73,826278.36,0.0009,0.1005,7.436510e+04,0.09,0.08,2.575251,11.92,0.8,0.26,AAA,0.999,22.10,261.54,208.33
54179,SUCURSAL BARRANQUILLA,DLS00457189,2025.0,40,26.304931,5268.78,138594.89,0.0001,0.5466,2.845961e+11,2053438.71,0.09,0.000000,2053450.77,0.0,0.24,AAA,0.999,67.08,303.83,251.22
54189,SUCURSAL BARRANQUILLA,DAB02772025,2025.0,40,24.894806,10883.85,270951.33,0.0003,0.3553,4.877120e+04,0.18,0.09,3.767901,2053452.06,1.0,0.40,AAA,0.999,52.46,276.52,226.73
54205,SUCURSAL BARRANQUILLA,DAB02666025,2025.0,40,22.608037,11371.65,257090.68,0.0003,0.3679,3.085090e+04,0.12,0.11,2.270270,2053454.55,0.8,0.38,AAA,0.999,23.55,227.02,181.80
54218,SUCURSAL BARRANQUILLA,BCS00060125,2025.0,40,21.692363,30382.98,659078.63,0.0007,0.1319,1.120434e+05,0.17,0.12,2.789855,2053456.57,0.8,0.30,AAA,0.999,30.72,225.95,182.57
54247,SUCURSAL BARRANQUILLA,DAB14772025,2025.0,40,19.561074,9112.40,178248.33,0.0002,0.4587,3.743210e+04,0.21,0.14,3.928058,2053460.94,1.0,0.47,AAA,0.999,34.93,210.98,171.86


# INVENTARIO ACTUAL

In [51]:
ruta = "/home/donsson/proyectos/INVENTARIO/inventario_limpio.xlsx"

inv = pd.read_excel(ruta)

inv["product_ref"] = inv["Complete name"].str.extract(r"\[([A-Z0-9]+)\]")

In [52]:
inv.head(10)

,Sucursal,Estado,Codigo_interno,Complete name,Clase,Marca,categoria_producto,Cantidad,Costo unitario,Costo total,product_ref
0,PRINCIPAL COTA,Existencias,004 DA4207,[DAE04207004] DA4207 FILTRO AIRE SEGURIDAD KOMATSU (004 AF574SY),C,AUT*PARTS,Filtro,1.0,1.00,1.00,DAE04207004
1,PRINCIPAL COTA,Existencias,004 G314,[DLE00314004] G314 FILTRO ACEITE ESPECIAL (004 F62H),C,AUT*PARTS,Filtro,2.0,1890.00,3780.00,DLE00314004
2,PRINCIPAL COTA,Existencias,004 G401,[DLE00401004] G401 FILTRO ACEITE FIAT-ALLIS 75215486 (004 F75215486),C,AUT*PARTS,Filtro,7.0,1.00,7.00,DLE00401004
3,PRINCIPAL COTA,Existencias,004 GA128,[DAE05128004] GA128 FILTRO AIRE FIAT DATSUN (004 GA128),C,AUT*PARTS,Filtro,1.0,1694.96,1694.96,DAE05128004
4,SUCURSAL MEDELLIN,Existencias,004 GA128,[DAE05128004] GA128 FILTRO AIRE FIAT DATSUN (004 GA128),C,AUT*PARTS,Filtro,2.0,1694.96,3389.92,DAE05128004
5,PRINCIPAL COTA,Existencias,004 GA158,[DAE05158004] GA158 FILTRO AIRE RENULT 4 Y 6. (004 GA158),C,AUT*PARTS,Filtro,1.0,1722.50,1722.50,DAE05158004
6,PRINCIPAL COTA,Existencias,004 GA182,[DAE05182004] GA182 FILTRO AIRE SIMCA 1000 Y 1300 (004 GA182),C,AUT*PARTS,Filtro,1.0,1462.50,1462.50,DAE05182004
7,PRINCIPAL COTA,Existencias,004 GA321,[DAE05321004] GA321 FILTRO AIRE MONZA (004 GA321),C,AUT*PARTS,Filtro,2.0,1716.00,3432.00,DAE05321004
8,SUCURSAL BUCARAGAMNGA,Existencias,004 GS050,"[DLS00050004] GS050 FILTRO ACEITE SWIFT,SPRINT. (004 GS050)",C,AUT*PARTS,Filtro,1.0,6293.12,6293.12,DLS00050004
9,SUCURSAL NORTE,Existencias,004 GS050,"[DLS00050004] GS050 FILTRO ACEITE SWIFT,SPRINT. (004 GS050)",C,AUT*PARTS,Filtro,1.0,6293.12,6293.12,DLS00050004


In [53]:
inv["Cantidad"].sum() 

np.float64(319577.719)

# EVALUACION Y COMPARACION

In [58]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

productos_ema = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["sucursal_id","product_name","ano","semana","ema","producto_costo_unitario","punto_reorden","clasificacion","almacen_id","inventario_seguridad","inventario_ideal" ]}
)

df = pd.DataFrame(productos_ema)

def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

df.rename(columns={"product_name": "product_ref"}, inplace=True)


df["store_id_num"] = df["sucursal_id"].apply(extract_id)
df["store_name"]   = df["sucursal_id"].apply(extract_name)

df["almacen_id_num"] = df["almacen_id"].apply(extract_id)
df["almacen_name"]   = df["almacen_id"].apply(extract_name)


In [59]:
df_40 = df_std[df_std["semana"]==40]

df_40["Clasificacion_mia"] = df_40["Clasificacion"]

/tmp/ipykernel_252495/3541219642.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_40["Clasificacion_mia"] = df_40["Clasificacion"]


In [60]:
# --- MERGE LIMPIO PARA COMPARAR ---
df_merge_simple = pd.merge(
    df_40[['store_name', 'product_ref', 'Clasificacion_mia', 
            'inventario_seguridad', 'inventario_ideal', 'punto_reorden']],
    df[['store_name', 'product_ref', 'clasificacion', 
         'inventario_seguridad', 'inventario_ideal', 'punto_reorden']],
    on=['store_name', 'product_ref'],
    how='inner',
    suffixes=('_std', '_real')
)

# --- COMPARACIÓN DE CLASIFICACIÓN ---
df_merge_simple['match_clasificacion'] = (
    df_merge_simple['Clasificacion_mia'] == df_merge_simple['clasificacion']
)

# --- OPCIONAL: ORDENAR PARA REVISAR FÁCIL ---
df_merge_simple = df_merge_simple.sort_values(by=['store_name', 'product_ref'])

# --- MOSTRAR SOLO LAS COLUMNAS RELEVANTES ---
cols = [
    'store_name', 'product_ref',
    'Clasificacion_mia', 'clasificacion', 'match_clasificacion',
    'inventario_seguridad_std', 'inventario_seguridad_real',
    'inventario_ideal_std', 'inventario_ideal_real',
    'punto_reorden_std', 'punto_reorden_real'
]

df_merge_simple = df_merge_simple[cols]



df_merge_simple.to_excel("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/diferencias_ema.xlsx")

In [63]:
df_merge_simple.sample(10)

,store_name,product_ref,Clasificacion_mia,clasificacion,match_clasificacion,inventario_seguridad_std,inventario_seguridad_real,inventario_ideal_std,inventario_ideal_real,punto_reorden_std,punto_reorden_real
16337,SUCURSAL MEDELLIN,BLS00259125,C,C,True,0.0,0.0,2.11,3,1.64,2
12065,SUCURSAL CALLE 6,DAB18178025,C,C,True,0.0,0.0,0.65,1,0.52,1
18437,SUCURSAL MEDELLIN,DAE09216025,C,C,True,0.0,0.0,1.46,2,1.14,2
7495,SUCURSAL CALI,BCS00367125,C,C,True,0.0,0.0,1.05,2,0.82,1
16202,SUCURSAL MEDELLIN,BLS00070125,C,C,True,0.0,0.0,0.00,0,0.00,0
4416,SUCURSAL BUCARAMANGA,BHE00806125,C,C,True,0.0,0.0,0.20,1,0.15,1
3495,SUCURSAL BUCARAMANGA,BAB08030125,C,C,True,0.0,0.0,0.00,0,0.00,0
22674,SUCURSAL NORTE,DAE06980115,C,C,True,0.0,0.0,0.00,0,0.00,0
21998,SUCURSAL NORTE,DAB08079025,C,C,True,0.0,0.0,0.00,0,0.00,0
14429,SUCURSAL MEDELLIN,BAP18046125,C,C,True,0.0,0.0,0.00,0,0.00,0
